## Введение в обработку естественного языка

Домашнее задание №2

Урок 2. Создание признакового пространства

*Формат именования файла домашней работы: FIO_NLP_HW_N.ipynb, где N - номер домашнего задания*

**

Задание 1.
Задание: обучите три классификатора:
- на токенах с высокой частотой
- на токенах со средней частотой
- на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

Задание 2.  
- найти фичи с наибольшей значимостью, и вывести их

Задание 3.
- сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)
- подобрать оптимальный размер для hashing векторайзера
- убедиться что для сетки нет переобучения

**Установка и импорт необходимых библиотек**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

### Задание 1

Загрузка данных

In [2]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2023-04-04 18:13:41--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2023-04-04 18:13:42--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaf52935ba0612ad69d927e5e41.dl.dropboxusercontent.com/cd/0/inline/B5hiy-aBc-87dJx6sYwe0I-iNAAJ5GQd3Mvfe3hr4TognW67MvokFZjY-3ugILl5x7hdEbCkRhwhD02IWt2_i62Y93iUWE6TF68Ge1NR5hLQZEwUrN_JgbDc2iZl2m4SFTUKRdoYnbab-e4pYdJ74v_qmcnTDEu9GWmqe4RAWofRNA/file# [following]
--2023-04-04 18:13:42--  https://ucaf52935ba0612ad69d927e5e41.dl.dropboxusercontent.com/cd/0/inline/B5hiy-aBc-87dJx6sYwe0I-iNAAJ5GQd3Mvfe3hr4TognW67MvokFZjY-3ugILl5x7hdEbCkRhwhD

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
my_stopwords = stopwords.words('russian')
noise = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

<ipython-input-4-67f5acc3628f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = positive.append(negative)


In [5]:
df

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive
...,...,...
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative


In [6]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [8]:
def get_tokens_importance(maxdf, mindf):
    
    tfidf_vect = TfidfVectorizer(max_df=maxdf, # не берем слова что выше трешхолда
                                 min_df=mindf, # не берем что ниже трешхолда
#                                  max_features=1000, 
#                                  stop_words=noise,
                                 )
    
    bow = tfidf_vect.fit_transform(x_train)
    clf = LogisticRegression(random_state=42)
    clf.fit(bow, y_train)
    
    pred = clf.predict(tfidf_vect.transform(x_test))
    print(f'max_df= {maxdf} min_df= {mindf}')
    print(classification_report(pred, y_test))

In [9]:
# Берем все токены - f1-score получается низкая
get_tokens_importance(1, 1)

max_df= 1 min_df= 1
              precision    recall  f1-score   support

    negative       0.90      0.53      0.66     47567
    positive       0.22      0.68      0.33      9142

    accuracy                           0.55     56709
   macro avg       0.56      0.60      0.50     56709
weighted avg       0.79      0.55      0.61     56709



In [10]:
# Берем токены часто встречающиеся
get_tokens_importance(1.0, 0.285)

max_df= 1.0 min_df= 0.285
              precision    recall  f1-score   support

    negative       0.36      0.61      0.45     16424
    positive       0.77      0.55      0.64     40285

    accuracy                           0.57     56709
   macro avg       0.56      0.58      0.55     56709
weighted avg       0.65      0.57      0.59     56709



In [11]:
# Берем токены средне встречающиеся
get_tokens_importance(0.7, 0.15)

max_df= 0.7 min_df= 0.15
              precision    recall  f1-score   support

    negative       0.35      0.61      0.44     16184
    positive       0.78      0.55      0.64     40525

    accuracy                           0.57     56709
   macro avg       0.56      0.58      0.54     56709
weighted avg       0.66      0.57      0.59     56709



In [12]:
# Берем токены низкочастотные
get_tokens_importance(0.3, 1)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


max_df= 0.3 min_df= 1
              precision    recall  f1-score   support

    negative       0.74      0.77      0.75     26994
    positive       0.78      0.75      0.77     29715

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



Основываясь на результатах метрик можно выжвинуть гипотезу о том, что низкочастотные токены наиболее важны для классификации

### Задание 2

In [13]:
from collections import Counter
from nltk.tokenize import word_tokenize

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
#corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
corpus = [token for tweet in df.text for token in word_tokenize(tweet)]
print(len(corpus))
corpus[:10]

4027088


['@', 'first_timee', 'хоть', 'я', 'и', 'школота', ',', 'но', 'поверь', ',']

In [16]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('(', 212404),
 (')', 194005),
 (',', 188295),
 (':', 177675),
 ('@', 149978),
 ('не', 69472),
 ('!', 66923),
 ('.', 57623),
 ('и', 55166),
 ('в', 52902)]

Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [17]:
cool_token = '!'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.87      0.51      0.64     48330
    positive       0.17      0.58      0.26      8379

    accuracy                           0.52     56709
   macro avg       0.52      0.54      0.45     56709
weighted avg       0.77      0.52      0.59     56709



In [18]:
cool_token = ':'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.48      0.54      0.51     24806
    positive       0.61      0.54      0.57     31903

    accuracy                           0.54     56709
   macro avg       0.54      0.54      0.54     56709
weighted avg       0.55      0.54      0.55     56709



In [19]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32956
    positive       0.83      1.00      0.91     23753

    accuracy                           0.91     56709
   macro avg       0.91      0.93      0.91     56709
weighted avg       0.93      0.91      0.91     56709



In [20]:
cool_token = '('
pred = ['negative' if cool_token in tweet else 'positive' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.95      1.00      0.97     26643
    positive       1.00      0.95      0.98     30066

    accuracy                           0.98     56709
   macro avg       0.98      0.98      0.98     56709
weighted avg       0.98      0.98      0.98     56709



Гипотеза:

")" Это часть смайлика или сам смайлик - и его присутствие в твите в основном говорит о положительном твите и наоборот.  
В свою очередь "(" это часть огорченного смайлика - и его присутствие - признак отризательного твита

### Задание 3

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
%%time

count_vect = CountVectorizer()
bow = count_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

pred = clf.predict(count_vect.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.77      0.76      0.76     28429
    positive       0.76      0.77      0.76     28280

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709

CPU times: user 13.7 s, sys: 9.35 s, total: 23 s
Wall time: 17.4 s


In [23]:
%%time

tfidf_vect = TfidfVectorizer()
bow = tfidf_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

pred = clf.predict(tfidf_vect.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.74      0.77      0.75     26994
    positive       0.78      0.75      0.77     29715

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709

CPU times: user 15.2 s, sys: 9.87 s, total: 25.1 s
Wall time: 17.4 s


In [24]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LogisticRegression

In [25]:
%%time

feature_sizes = [100, 1000, 10000, 100000, 1000000]

for feature_size in feature_sizes:
    
    h_vect = HashingVectorizer(n_features=feature_size)
    h_vect.fit(x_train)

    xtrain_count =  h_vect.transform(x_train)
    xtest_count =  h_vect.transform(x_test)

    classifier = LogisticRegression()
    classifier.fit(xtrain_count, y_train)
    predictions = classifier.predict(xtest_count)
    #predictions
    print(f'Длина hash-вектора: {feature_size}')
    print(classification_report(predictions, y_test))

Длина hash-вектора: 100
              precision    recall  f1-score   support

    negative       0.58      0.59      0.58     27159
    positive       0.62      0.60      0.61     29550

    accuracy                           0.60     56709
   macro avg       0.60      0.60      0.60     56709
weighted avg       0.60      0.60      0.60     56709

Длина hash-вектора: 1000
              precision    recall  f1-score   support

    negative       0.63      0.65      0.64     27348
    positive       0.67      0.65      0.66     29361

    accuracy                           0.65     56709
   macro avg       0.65      0.65      0.65     56709
weighted avg       0.65      0.65      0.65     56709



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Длина hash-вектора: 10000
              precision    recall  f1-score   support

    negative       0.70      0.71      0.71     27416
    positive       0.73      0.71      0.72     29293

    accuracy                           0.71     56709
   macro avg       0.71      0.71      0.71     56709
weighted avg       0.71      0.71      0.71     56709



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Длина hash-вектора: 100000
              precision    recall  f1-score   support

    negative       0.72      0.75      0.74     27116
    positive       0.76      0.74      0.75     29593

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Длина hash-вектора: 1000000
              precision    recall  f1-score   support

    negative       0.73      0.76      0.74     26942
    positive       0.77      0.74      0.76     29767

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709

CPU times: user 54.9 s, sys: 28.3 s, total: 1min 23s
Wall time: 58.2 s


In [26]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout
from tensorflow.keras.layers import TextVectorization # формирует словарь как CountVect и TF-IDF 
# из унгикальных токенов и заменяет кажддый токен его айдишником 
from sklearn.preprocessing import LabelEncoder

In [27]:
# Приведем y к 0,1

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [28]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [29]:
train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [30]:
for raw in train_data.take(1):
    pass

In [31]:
raw

(<tf.Tensor: shape=(16,), dtype=string, numpy=
 array([b'RT @KressVika: \xd0\x9f\xd0\xb0\xd1\x80\xd0\xbd\xd0\xb8 \xd0\xb8\xd0\xb7 \xd1\x84\xd0\xb8\xd0\xb7-\xd0\xbc\xd0\xb0\xd1\x82\xd0\xb0 \xd0\xbb\xd1\x83\xd1\x87\xd1\x88\xd0\xb8\xd0\xb5, \xd1\x83\xd1\x80\xd0\xbe\xd0\xba \xd0\xbd\xd0\xb0 \xd1\x83\xd1\x80\xd0\xb0)',
        b'RT @Abyssal_Blade: @alessen_sun \n\xd0\xba\xd0\xb0\xd0\xba \xd0\xb6\xd0\xb5 \xd1\x82\xd1\x8b \xd0\xb1\xd0\xb5\xd0\xb7 \xd0\xba\xd0\xb0\xd1\x80\xd0\xbc\xd0\xb0\xd0\xbd\xd0\xbd\xd0\xbe\xd0\xb3\xd0\xbe \xd1\x88\xd1\x83\xd1\x82\xd0\xb0?(',
        b'\xd0\x94\xd0\xbe \xd1\x81\xd0\xbb\xd1\x91\xd0\xb7 \xd0\xb1\xd0\xbb\xd0\xb8\xd0\xbd \xd0\xb7\xd0\xb0\xd0\xbf\xd0\xb8\xd1\x81\xd1\x8c \xd0\xb4\xd0\xbe\xd0\xb2\xd0\xb5\xd0\xbb\xd0\xb0 ,\xd0\xb0\xd1\x85\xd0\xb0\xd1\x85\xd0\xb0\xd1\x85)))\n4 \xd1\x80\xd0\xb0\xd0\xb7\xd0\xb0 \xd0\xbf\xd0\xbb\xd0\xb0\xd0\xba\xd0\xb0\xd0\xbb\xd0\xb0 =))) http://t.co/p259gl1f2s',
        b'RT @nebo_oben: @pg__lost \xd0\xbd\xd0\xb5\xd1\x82, \xd1\x8d\x

In [32]:
def custom_standardization(input_data):
    
    return input_data

vocab_size = 10000 # выбрали 10000 а там было 31к
seq_len = 100 # сколько токенов в тексте -проходят по всем текстам и выбирают максимум, или берут квантильное - 
# могут быть выбросы по длинам 

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int', # что каждый токен будем переводить в индекс относительно нашего словаря vocab_size
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

embedding_dim=200 # 200-мерные вектора будут у каждого токена

In [33]:
class myNet(tf.keras.Model):
    def __init__(self):
        super(myNet, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim, name="embedding")
        self.conv1 = Conv1D(200, (3))
        self.conv2 = Conv1D(200, (3))
        self.gPool = GlobalAveragePooling1D()
        self.fc1 = Dense(100, activation='relu')
        self.fc2 = Dense(1)


    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        x1 = self.conv1(x)
        x = self.conv2(x)
        x = self.gPool((x + x1)/2)
        x = self.fc1(x)
#         x = self.ss(x)
        return self.fc2(x)
    
# тут подход что все выделено в класс - ти удобнее писать в таком подходе!!!!
# так как еслп будут повторться слои 

In [34]:
mmodel = myNet()

In [35]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [36]:
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
10633/10633 [==============================] - 98s 8ms/step - loss: 0.3893 - accuracy: 0.7753 - val_loss: 0.3479 - val_accuracy: 0.7951
Epoch 2/5
10633/10633 [==============================] - 70s 7ms/step - loss: 0.3276 - accuracy: 0.8189 - val_loss: 0.3403 - val_accuracy: 0.8100
Epoch 3/5
10633/10633 [==============================] - 69s 6ms/step - loss: 0.3053 - accuracy: 0.8335 - val_loss: 0.3450 - val_accuracy: 0.8057
Epoch 4/5
10633/10633 [==============================] - 68s 6ms/step - loss: 0.2848 - accuracy: 0.8469 - val_loss: 0.3736 - val_accuracy: 0.7972
Epoch 5/5
10633/10633 [==============================] - 68s 6ms/step - loss: 0.2617 - accuracy: 0.8618 - val_loss: 0.4066 - val_accuracy: 0.8129


Переобучение уже пошло с 3-й эпохи - и дальше стало только усиливаться

In [37]:
preds = mmodel.predict(valid_data)

3545/3545 [==============================] - 7s 2ms/step


In [38]:
preds

array([[ 22.623535 ],
       [ -0.5340227],
       [ 18.50459  ],
       ...,
       [  0.4341001],
       [ -7.740841 ],
       [-28.200327 ]], dtype=float32)

In [39]:
# Приведем preds в бинарный вид
preds = [int(i[0]>0) for i in preds]

In [40]:
# pred = clf.predict(tfidf_vect.transform(x_test))
print(classification_report(preds, y_test))

              precision    recall  f1-score   support

           0       0.80      0.83      0.82     27191
           1       0.84      0.81      0.82     29518

    accuracy                           0.82     56709
   macro avg       0.82      0.82      0.82     56709
weighted avg       0.82      0.82      0.82     56709

